
# Processamento de Dados com AWS S3 e Pandas

Este notebook contém as etapas necessárias para:
1. Carregar um arquivo CSV armazenado no **Amazon S3** diretamente no **Pandas**.
2. Limpar e processar os dados, padronizando valores e removendo inconsistências.
3. Realizar análises estatísticas sobre os dados processados.
4. Salvar os resultados das análises em um arquivo `.txt`.

## Pré-requisitos

### 1. Criar um ambiente virtual
1. Criar um ambiente virtual para gerenciar as dependências:
    ```bash
    python -m venv venv
    ```
2. Ativar o ambiente virtual:
    - **Windows:**  
      ```bash
      venv\Scripts\activate
      ```
    - **macOS/Linux:**  
      ```bash
      source venv/bin/activate
      ```
3. Instalar as bibliotecas necessárias:
    ```bash
    pip install boto3 pandas python-dotenv
    ```

### 2. Configurar o arquivo `.env`
Criar um arquivo `.env` na mesma pasta do notebook contendo as credenciais da AWS:
```
AWS_ACCESS_KEY_ID=your_aws_access_key_id
AWS_SECRET_ACCESS_KEY=your_aws_secret_access_key
AWS_SESSION_TOKEN=your_aws_session_token
REGION_NAME=us-east-1
```

### 3. Selecionar o Kernel no Jupyter Notebook
No Jupyter, vá até **Kernel** > **Change Kernel** e selecione **Python (venv)**.

Cada etapa está separada em células para facilitar a execução.


In [1]:

import pandas as pd
import os
import boto3
from dotenv import load_dotenv
from io import StringIO
import unicodedata
# Carregar variáveis de ambiente do .env
load_dotenv()

# Configurações do bucket
bucket_name = 'meu-bucket-sprint-4'
file_key = 'consumo-energetico-por-campus.csv'


## Etapa 1: Carregar Dados do S3
Esta função lê o arquivo CSV diretamente do S3 e retorna um DataFrame do Pandas.


In [2]:

# Função para carregar dados diretamente do S3 usando boto3
def carregar_dados_s3(bucket_name, file_key):
    """
    Carrega os dados diretamente do S3 usando boto3, sem baixar o arquivo.
    """
    try:
        s3 = boto3.client(
            's3',
            aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
            aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
            aws_session_token=os.getenv('AWS_SESSION_TOKEN')
        )
        response = s3.get_object(Bucket=bucket_name, Key=file_key)
        data = response['Body'].read().decode('utf-8')
        df = pd.read_csv(StringIO(data), delimiter=';')
        return df
    except Exception as e:
        print(f"Erro ao carregar dados do S3: {e}")
        raise


## Etapa 2: Limpeza dos Dados
Limpa e converte os dados do DataFrame.

Para as colunas numéricas (valor_fatura, quantidade_conceito_faturado, perda_transformacao):
- Converte os valores para string.
- Substitui vírgulas por pontos (para separador decimal).
- Remove caracteres que não sejam dígitos ou ponto.
- Converte para float, transformando entradas inválidas em NaN.
(Manipulação: Função de conversão)

Para as colunas de texto (como localidade e tipo_conceito_faturado):
- Converte os valores para string.
- Remove acentuação e caracteres especiais usando normalização Unicode.
- Converte para maiúsculas para padronização.
(Manipulação: Função de string)

Ao final, remove linhas duplicadas, mantendo apenas registros distintos.
(Manipulação: Remoção de duplicatas)


In [3]:
def limpar_dados(df):
    # Limpeza de colunas numéricas
    colunas_numericas = ['valor_fatura', 'quantidade_conceito_faturado', 'perda_transformacao']
    for coluna in colunas_numericas:
        df[coluna] = (
            df[coluna]
            .astype(str)
            .str.replace(',', '.', regex=False)  # Troca vírgula por ponto
            .str.replace(r'[^0-9\.]', '', regex=True)  # Remove caracteres não numéricos (exceto ponto)
        )
        # Converte para float; valores inválidos serão transformados em NaN
        df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
    
    # Limpeza de colunas de texto
    colunas_texto = []
    if 'localidade' in df.columns:
        colunas_texto.append('localidade')
    if 'tipo_conceito_faturado' in df.columns:
        colunas_texto.append('tipo_conceito_faturado')
    
    for coluna in colunas_texto:
        df[coluna] = df[coluna].astype(str).apply(
            lambda x: unicodedata.normalize('NFKD', x)   # Decompõe caracteres acentuados
                      .encode('ASCII', 'ignore')          # Remove acentos e caracteres especiais
                      .decode('utf-8')                    # Decodifica para string
                      .upper()                           # Converte para maiúsculas
        )
    
    # Remove linhas duplicadas (100% iguais)
    df = df.drop_duplicates()
    
    return df



## Etapa 3: Análises Estatísticas
Aqui são realizadas três análises:
1. **Top 10 locais com maior gasto total.**
2. **Top 10 locais mais baratos por custo médio.**
3. **Ranking de anos por média de valor faturado.**


In [4]:

def realizar_analises(df):
    resultados = []
    
    # ---- Análise 1: Top 10 locais por gasto total (valor_fatura) ----
    # Remove registros com localidade nula, vazia ou igual a "NAN" (após converter para uppercase).
    df_clean = df.dropna(subset=['localidade']).copy()
    df_clean = df_clean[df_clean['localidade'].astype(str).str.strip().str.upper() != "NAN"]
    
    # Filtra registros com valor_fatura > 1000 e quantidade_conceito_faturado > 50.
    df_a1 = df_clean[(df_clean['valor_fatura'] > 1000) & (df_clean['quantidade_conceito_faturado'] > 50)]
    # Agrupa por localidade e soma o valor_fatura.
    gasto_por_local = df_a1.groupby('localidade')['valor_fatura'].sum().sort_values(ascending=False)
    
    resultados.append("1. Top 10 locais por gasto total (valor_fatura):\n")
    for local, gasto in gasto_por_local.head(10).items():
        resultados.append(f"   - {local}: R$ {gasto:,.2f}\n")
    
    # ---- Análise 2: Top 10 locais mais baratos por custo médio de consumo ----
    # Seleciona registros com quantidade_conceito_faturado > 0 e localidade válida.
    df_a2 = df_clean[df_clean['quantidade_conceito_faturado'] > 0].copy()
    # Converte valor_fatura e quantidade_conceito_faturado para float.
    df_a2['valor_fatura'] = df_a2['valor_fatura'].astype(float)
    df_a2['quantidade_conceito_faturado'] = df_a2['quantidade_conceito_faturado'].astype(float)
    # Agrupa por localidade: soma valor_fatura e quantidade.
    grupo = df_a2.groupby('localidade').agg({
        'valor_fatura': 'sum',
        'quantidade_conceito_faturado': 'sum'
    })
    # Calcula o custo médio de consumo para cada local.
    grupo['custo_medio'] = grupo['valor_fatura'] / grupo['quantidade_conceito_faturado']
    overall_avg = grupo['custo_medio'].mean()
    # Classifica: "CUSTO BAIXO" se custo_medio < média geral; senão "CUSTO ALTO".
    grupo['classificacao'] = grupo['custo_medio'].apply(lambda x: "CUSTO BAIXO" if x < overall_avg else "CUSTO ALTO")
    # Ordena do menor para o maior custo médio.
    grupo = grupo.sort_values('custo_medio', ascending=True)
    
    resultados.append("\n2. Top 10 locais mais baratos por custo médio de consumo (valor_fatura / quantidade):\n")
    for local, row in grupo.head(10).iterrows():
        resultados.append(f"   - {local}: R$ {row['custo_medio']:,.2f} ({row['classificacao']})\n")
    
    # ---- Análise 3: Ranking de anos por média de valor_fatura ----
    # Remove registros com ano nulo ou vazio.
    df_a3 = df.dropna(subset=['ano']).copy()
    df_a3 = df_a3[df_a3['ano'].astype(str).str.strip() != ""]
    # Agrupa por ano e calcula a média de valor_fatura.
    media_por_ano = df_a3.groupby('ano')['valor_fatura'].mean().sort_values(ascending=False)
    
    resultados.append("\n3. Ranking de anos por média de valor_fatura:\n")
    for ano, media in media_por_ano.head(10).items():
        resultados.append(f"   - {ano}: R$ {media:,.2f}\n")
    
    return resultados



## Etapa 4: Salvar Resultados
Os resultados são salvos em um arquivo `.txt`.


In [5]:
# Função para salvar resultados em um arquivo TXT
def salvar_resultados(resultados, output_file):
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("=== Resultados das Análises ===\n\n")
            f.writelines(resultados)
        print(f"Resultados salvos em {output_file}")
    except Exception as e:
        print(f"Erro ao salvar resultados: {e}")


## Etapa 5: Execução do Processamento
Esta etapa **automatiza o processamento dos dados** extraídos do **Amazon S3**, garantindo que todas as transformações sejam aplicadas corretamente. O fluxo é projetado para rodar **de ponta a ponta**, sem necessidade de intervenção manual.



In [6]:
print("Iniciando o processamento da Etapa 2...")

try:
    # Carregar dados do S3
    print(f"Lendo dados diretamente do bucket '{bucket_name}'...")
    df = carregar_dados_s3(bucket_name, file_key)
    print("Dados carregados com sucesso!")

    # Limpar e converter os dados
    print("Limpando os dados...")
    df = limpar_dados(df)

    # Realizar análises
    print("Realizando análises...")
    resultados = realizar_analises(df)

    # Salvar os resultados
    salvar_resultados(resultados, 'resultados_analises_etapa2.txt')

except Exception as e:
    print(f"Erro no processamento: {e}")

print("Processamento da Etapa 2 concluído.")

Iniciando o processamento da Etapa 2...
Lendo dados diretamente do bucket 'meu-bucket-sprint-4'...


C:\Users\Vini\AppData\Local\Temp\ipykernel_64556\2044927699.py:15: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(data), delimiter=';')


Dados carregados com sucesso!
Limpando os dados...
Realizando análises...
Resultados salvos em resultados_analises_etapa2.txt
Processamento da Etapa 2 concluído.
